In [1]:
# from functions.satellites_ephemeris import SatelliteEphemeris

# filename = "functions/ephemerides/tles50.json"
# ephemeris = SatelliteEphemeris(filename)
# ephemeris.load_data()
# debris_data = ephemeris.get_debris()

# print(f"Number of objects in DB: {len(debris_data)} \n")

Custom database loaded: functions/ephemerides/tles50.json
Number of objects in DB: 50 



In [1]:
from functions.satellites_ephemeris import SatelliteEphemeris

filename = 'functions/ephemerides/OneWeb_Sat_Data.xlsx'
ephemeris = SatelliteEphemeris(filename=filename)
ephemeris.load_data()
debris_data = ephemeris.get_debris()

Custom database loaded: functions/ephemerides/OneWeb_Sat_Data.xlsx


In [2]:
from functions.MRPLP_J2_analytic import MultiRevolutionPerturbedLambertSolver
import numpy as np

t1 = 23902.49
id1 = '45'

t2 = 23919.56-0.5
id2 = '44'

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

tof = ( t2 - t1 ) * 86400

mu = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

MRPLPsolver = MultiRevolutionPerturbedLambertSolver() # MRPLP solver

# set the parameters for the MRPLP solver
order = 7
parameters = MRPLPsolver.mrplp_J2_analytic_parameters(cart1[0:3], cart2[0:3], tof, cart1[3:], mu, Re, J2,
                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, scl=1.0e-3, itermax=200 )

# solve the MRPLP
output = MRPLPsolver.mrplp_J2_analytic(parameters)

# compute the DV
dvv1 = output.vv1Sol - cart1[3:]
dvv2 = cart2[3:] - output.vv2Sol
dv1 = np.linalg.norm( dvv1 )
dv2 = np.linalg.norm( dvv2 )
dvtot = dv1 + dv2

# print the output
print(f"-------------------------------------------------------")
print(f"                 OUTPUT SUMMARY")
print(f"-------------------------------------------------------")
print(f"Order of the expansion  : {order}")
print(f"Success                 : {output.success}")
print(f"Elapsed time            : {output.elapsed_time} seconds")
print(f"Final pos. error (norm) : {np.linalg.norm( output.rr2DA - cart2[0:3] )} m")
print(f"-------------------------------------------------------")
print(f"Delta_v1                : {dv1} m/s")
print(f"Delta_v2                : {dv2} m/s")
print(f"Delta_vtot              : {dvtot} m/s")
print(f"-------------------------------------------------------")


-------------------------------------------------------
                 OUTPUT SUMMARY
-------------------------------------------------------
Order of the expansion  : 7
Success                 : True
Elapsed time            : 0.4691321849822998 seconds
Final pos. error (norm) : 1.7310617483236337e-06 m
-------------------------------------------------------
Delta_v1                : 210.99851623157832 m/s
Delta_v2                : 234.02895677271098 m/s
Delta_vtot              : 445.0274730042893 m/s
-------------------------------------------------------


In [10]:
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2

start = 8487.0
end = start + 30
step = 1.0
t1_list = np.arange(start, end + step, step)

id_start = [str(i) for i in range(1, len(debris_data))]
id_tar = [str(i) for i in range(1, len(debris_data))]

########## START: define the DV options ##########
dv_tol_an = 1500
dv_tol = 200
########## END: define the DV options ##########

########## START: define the TOF options ##########
start_tof = 5.0 # days
end_tof = 30.0 # days
grid_size = 2.0 # days
tof_list = np.arange(start_tof, end_tof + grid_size, grid_size)
########## END: define the TOF options ##########

for id1 in id_start:

    for t1 in t1_list:

        for id2 in id_tar:

            if id1 != id2:

                for tof in tof_list:

                    t2 = t1 + tof
                    tof_secs = tof * 86400.0

                    cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
                    cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )
                    vv1g = cart1[3:]

                    # maybe first do an approximate DV computation, and then use the MRPLP
                    paramsAnalytical = AnalyticDVApproxJ2.approx_DV_parameters( cart1[0:3], cart1[3:],
                                                                                cart2[0:3], cart2[3:],
                                                                                t1, t2,
                                                                                mu, Re, J2)
                    delta_v = AnalyticDVApproxJ2.approx_DV(paramsAnalytical)

                    if delta_v <= dv_tol_an:

                        # set the parameters for the MRPLP solver
                        order = 7
                        parameters = MRPLPsolver.mrplp_J2_analytic_parameters(cart1[0:3], cart2[0:3], tof_secs, vv1g, mu, Re, J2,
                                                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, 
                                                                            scl=1.0e-3, itermax=150 )
                        
                        # solve the MRPLP
                        output = MRPLPsolver.mrplp_J2_analytic(parameters)
                        
                        # compute the DV
                        dvv1 = output.vv1Sol - cart1[3:]
                        dvv2 = cart2[3:] - output.vv2Sol
                        dv1 = np.linalg.norm( dvv1 )
                        dv2 = np.linalg.norm( dvv2 )
                        dvtot = dv1 + dv2

                        # print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, DV: {delta_v} m/s - approx. DV")
                        if np.isnan(dvtot):
                            """ print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION - NaN") """
                        else:
                            if dvtot < dv_tol:
                                print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, DV: {dvtot} m/s")
                            else:
                                """ print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION") """
                    else:
                        """ print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION - approx. DV: {delta_v} m/s") """


KeyboardInterrupt: 

In [ ]:
# analytic computation of DV
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2
import numpy as np

mue = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

id1 = '0'
id2 = '302'
t1 = 8487.0
t2  = t1 + 10.0

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

# Initialize parameters
parameters = AnalyticDVApproxJ2.approx_DV_parameters(
    rr1=cart1[0:3],
    vv1=cart1[3:],
    rr2=cart2[0:3],
    vv2=cart2[3:],
    t1=t1,
    t2=t2,
    mu=mue,
    rE=Re,
    J2=J2
)

delta_v = AnalyticDVApproxJ2.approx_DV(parameters)
delta_v